In [1]:
import numpy as np

def my_matmul(mat_list):
    # mat_list is an iterable of 1) matrices or 2) field of matrices
    return reduce(lambda x, y: np.matmul(x, y), mat_list)
    
def adj(a): # hermitian conjugate on last two axis
    return np.conj(np.moveaxis(a, -2, -1)) # transpose doesn't work; so used moveaxis

def trace(a): # race on last two axis
    return np.trace(a, axis1=-2, axis2=-1)

#  Grid/documentation/manual.rst 
#  Grid/documentation/Grid.pdf
# \gamma_x= \left(\begin{array}{cccc}    0& 0& 0& i\\  0& 0& i& 0\\  0&-i& 0& 0\\ -i& 0& 0& 0 \end{array}\right)
# \gamma_y= \left(\begin{array}{cccc}    0& 0& 0&-1\\  0& 0& 1& 0\\  0& 1& 0& 0\\ -1& 0& 0& 0 \end{array}\right)
# \gamma_z= \left(\begin{array}{cccc}    0& 0& i& 0\\  0& 0& 0&-i\\ -i& 0& 0& 0\\  0& i& 0& 0 \end{array}\right)
# \gamma_t= \left(\begin{array}{cccc}    0& 0& 1& 0\\  0& 0& 0& 1\\  1& 0& 0& 0\\  0& 1& 0& 0 \end{array}\right)
# \gamma_5= \left(\begin{array}{cccc}    1& 0& 0& 0\\  0& 1& 0& 0\\  0& 0&-1 &0\\  0& 0& 0&-1 \end{array}\right)

# https://en.wikipedia.org/wiki/Gamma_matrices#Euclidean_Dirac_matrices
# p.s. this is the same as Bigeng's convention 
# /home/bw2482/Research/2020/Free_Field_Inami_Lim_check/v3_heavy_up/Gmat.m

gamma_mu = np.array([
    [[0,0,0,1j], 
     [0,0,1j,0], 
     [0,-1j,0,0], 
     [-1j,0,0,0]], # gamma_x
    
    [[0,0,0,-1], 
     [0,0,1,0], 
     [0,1,0,0], 
     [-1,0,0,0]], # gamma_y
    
    [[0,0,1j,0], 
     [0,0,0,-1j], 
     [-1j,0,0,0], 
     [0,1j,0,0]], # gamma_z
    
    [[0,0,1,0],
     [0,0,0,1], 
     [1,0,0,0], 
     [0,1,0,0]], # gamma_t
])
gamma5 = np.array(
    [[1,0,0,0], 
     [0,1,0,0], 
     [0,0,-1,0], 
     [0,0,0,-1]],
dtype='complex128')

# Check gamma1 * gamma2 * gamma3 * gamm4 == gamma5
from functools import reduce
assert np.allclose(my_matmul(gamma_mu),  gamma5)
assert trace(np.matmul(gamma_mu[1], gamma_mu[1])) == 4

In [2]:
m_f = 0.06
M5 = 1.0

# lat_size = np.array([4,4,4,4])
LAT_SIZE = np.array([8,8,8,8])
# LAT_SIZE = np.array([16,16,16,16])



# Point source propagator

In [3]:
def calc_point_prop(M5, m_f, LAT_SIZE):

    a_5 = 1.0 
    a = 1.0
    eta = a_5 / a

    r = 1.00
    
    VOL = np.prod(LAT_SIZE)
    
    # coor = (coor[0], ... coor[3]);  coor[0] : the x coordinate of all sites
    coor = np.meshgrid(*[range(x) for x in LAT_SIZE], indexing='ij') # shape: (4, Lx, Ly, Lz, Lt)
    pmu_a = 2 * np.pi / LAT_SIZE.reshape(4,1,1,1,1) * coor  # p.s. pmu_a is dimensionaless
 
    W = (1 - M5) + r * eta * np.sum(1 - np.cos(pmu_a), axis=0) # W shape: (Lx, Ly, Lz, Lt)

    x = eta**2 * np.sum(np.sin(pmu_a)**2, axis=0)
    alpha = np.arccosh(  ((1 + W**2 + x) / (2 * W)).astype(dtype=np.complex)  )  # Taku Eq. 42

    # fp(0,0,0,0) = inf; will be RuntimeWarning: divide by zero encountered in true_divide; 
    # We deal with the zero mode separately. It is safe to set fp(0,0,0,0) to 0, and add another term.
    f_p = 1 / (W * np.exp(alpha) - 1 + m_f**2 * (1 - W * np.exp(-alpha))) 
    f_p[0,0,0,0] = 0

    cmu_p = -1j * eta * np.sin(pmu_a)
    c1_p = - m_f * (1 - W * np.exp(-alpha))

    Sx_I_coef = np.fft.ifftn(f_p * c1_p) - 1/m_f/VOL # - 1/m_f/VOL comes from the zero mode
    Sx_gmu_coef = [np.fft.ifftn(f_p * cmu_p[mu]) for mu in range(4)]

    Sx = Sx_I_coef[..., None, None] * np.eye(4) + \
         sum([Sx_gmu_coef[mu][..., None, None] * gamma_mu[mu] for mu in range(4)])
    
    return Sx

In [5]:
# gamma5 hermiticity test

Sx = calc_point_prop(M5=M5, m_f=m_f, LAT_SIZE=LAT_SIZE)
Sx.real[abs(Sx.real) < 1e-10] = 0   # Set very small values to 0
Sx.imag[abs(Sx.imag) < 1e-10] = 0   # Set very small values to 0

tmp = my_matmul([gamma5, adj(Sx), gamma5])
for site in [(0,0,0,0), (1,1,0,1), (0,1,2,3)]:
    minus_site = tuple(-x for x in site)
    print(site, minus_site)
    assert np.allclose(Sx[minus_site], tmp[site])    

(0, 0, 0, 0) (0, 0, 0, 0)
(1, 1, 0, 1) (-1, -1, 0, -1)
(0, 1, 2, 3) (0, -1, -2, -3)


<ipython-input-3-282614f462ed>:18: RuntimeWarning: divide by zero encountered in true_divide
  alpha = np.arccosh(  ((1 + W**2 + x) / (2 * W)).astype(dtype=np.complex)  )  # Taku Eq. 42
<ipython-input-3-282614f462ed>:22: RuntimeWarning: invalid value encountered in multiply
  f_p = 1 / (W * np.exp(alpha) - 1 + m_f**2 * (1 - W * np.exp(-alpha)))


In [6]:
Sx[1,2,3,4]

array([[-0.00402012+0.j        ,  0.        +0.j        ,
         0.        +0.00013551j, -0.00017011+0.00013735j],
       [ 0.        +0.j        , -0.00402012+0.j        ,
         0.00017011+0.00013735j,  0.        -0.00013551j],
       [ 0.        -0.00013551j,  0.00017011-0.00013735j,
        -0.00402012+0.j        ,  0.        +0.j        ],
       [-0.00017011-0.00013735j,  0.        +0.00013551j,
         0.        +0.j        , -0.00402012+0.j        ]])

# Wall source propagator

In [7]:
def calc_wall_prop(M5, m_f, LAT_SIZE):
    
    L_T = LAT_SIZE[3]

    a_5 = 1.0
    a = 1.0
    eta = a_5 / a

    r = 1.00
    
    pt_a = 2 * np.pi / L_T * np.arange(L_T)  # p.s. pmu_a is dimensionaless

    W = (1 - M5) + r * eta * (1 - np.cos(pt_a)) # W shape: (Lx, Ly, Lz, Lt)

    x = eta**2 * np.sin(pt_a)**2
    alpha = np.arccosh(  ((1 + W**2 + x) / (2 * W)).astype(dtype=np.complex)  )  # Taku Eq. 42

    # fp(0,0,0,0) = inf; will be RuntimeWarning: divide by zero encountered in true_divide; 
    # We deal with the zero mode separately. It is safe to set fp(0,0,0,0) to 0, and add another term.
    f_p = 1 / (W * np.exp(alpha) - 1 + m_f**2 * (1 - W * np.exp(-alpha))) 
    f_p[0] = 0

    cmu_pt = -1j * eta * np.sin(pt_a)
    c1_p = - m_f * (1 - W * np.exp(-alpha))

    St_I_coef = np.fft.ifftn(f_p * c1_p) - 1/m_f/L_T # - 1/m_f/VOL comes from the zero mode
    St_gamma4_coef = np.fft.ifftn(f_p * cmu_pt)

    St = St_I_coef[..., None, None] * np.eye(4) + \
            St_gamma4_coef[..., None, None] * gamma_mu[3]
    
    return St
    

In [8]:
# gamma5 hermiticity test

St = calc_wall_prop(M5=M5, m_f=m_f, LAT_SIZE=LAT_SIZE)
St.real[abs(St.real) < 1e-10] = 0   # Set very small values to 0
St.imag[abs(St.imag) < 1e-10] = 0   # Set very small values to 0

tmp = my_matmul([gamma5, adj(St), gamma5])
for site in range(0, LAT_SIZE[3]):
    minus_site = -site
    print(site, minus_site)
    assert np.allclose(St[minus_site], tmp[site])    

0 0
1 -1
2 -2
3 -3
4 -4
5 -5
6 -6
7 -7


<ipython-input-7-4e986669fb45>:16: RuntimeWarning: divide by zero encountered in true_divide
  alpha = np.arccosh(  ((1 + W**2 + x) / (2 * W)).astype(dtype=np.complex)  )  # Taku Eq. 42
<ipython-input-7-4e986669fb45>:20: RuntimeWarning: invalid value encountered in multiply
  f_p = 1 / (W * np.exp(alpha) - 1 + m_f**2 * (1 - W * np.exp(-alpha)))


In [9]:
for t in range(LAT_SIZE[3]):
    print('t =', t)
    print(St[t])

t = 0
[[-2.11671812+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j -2.11671812+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j -2.11671812+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j -2.11671812+0.j]]
t = 1
[[-2.10133284+0.j  0.        +0.j  0.43672424+0.j  0.        +0.j]
 [ 0.        +0.j -2.10133284+0.j  0.        +0.j  0.43672424+0.j]
 [ 0.43672424+0.j  0.        +0.j -2.10133284+0.j  0.        +0.j]
 [ 0.        +0.j  0.43672424+0.j  0.        +0.j -2.10133284+0.j]]
t = 2
[[-2.07761171+0.j  0.        +0.j  0.26378414+0.j  0.        +0.j]
 [ 0.        +0.j -2.07761171+0.j  0.        +0.j  0.26378414+0.j]
 [ 0.26378414+0.j  0.        +0.j -2.07761171+0.j  0.        +0.j]
 [ 0.        +0.j  0.26378414+0.j  0.        +0.j -2.07761171+0.j]]
t = 3
[[-2.06533383+0.j  0.        +0.j  0.12813158+0.j  0.        +0.j]
 [ 0.        +0.j -2.06533383+0.j  0.        +0.j  0.12813158+0.j]
 [ 0.12813158+0.j  0.        +0.j -

# EM factor

In [11]:
def EM_factor(M_K, LAT_SIZE):
    r = np.meshgrid(*[range(x) for x in LAT_SIZE], indexing='ij') # shape: (4, Lx, Ly, Lz, Lt)

    for mu in range(4):  # IMPORTANT: if coor is greater than L/2, MUST map to coor - L.
        L = LAT_SIZE[mu]
        r[mu] = np.where(r[mu] <= L//2, r[mu], r[mu] - L)

    r3d_norm = np.sqrt(sum([r[i]**2 for i in range(3)])) # |vec{r}|
    rt = r[3]   # r_t

    tmp = np.exp(M_K * rt / 2) \
            * (  - M_K * r3d_norm * np.cos(M_K * r3d_norm / 2) + 2 * np.sin(M_K * r3d_norm / 2)  ) \
            / np.power(r3d_norm, 3)

    tmp[0,0,0] = 0.

    Euv = np.zeros((*LAT_SIZE, 4,4))

    Euv[..., 0, 1] = r[2] * tmp
    Euv[..., 0, 2] = - r[1] * tmp
    Euv[..., 1, 2] = r[0] * tmp
    Euv[..., 1, 0] = - Euv[..., 0, 1]
    Euv[..., 2, 0] = - Euv[..., 0, 2]
    Euv[..., 2, 1] = - Euv[..., 1, 2]
    return Euv

M_K = 0.5  # In C++, set M_K to the same value !!!
Euv = EM_factor(M_K=M_K, LAT_SIZE=LAT_SIZE)

<ipython-input-11-8434f0165444>:11: RuntimeWarning: invalid value encountered in true_divide
  tmp = np.exp(M_K * rt / 2) \


In [12]:
Euv[6,4,3,2]

array([[ 0.        ,  0.0427687 , -0.05702493,  0.        ],
       [-0.0427687 ,  0.        , -0.02851247,  0.        ],
       [ 0.05702493,  0.02851247,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])